#Libraries and Data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#change directory
%cd /content/drive/MyDrive/Time Series Forecasting Product

/content/drive/MyDrive/Time Series Forecasting Product


In [ ]:
#installing libraries
!pip install pyyaml==5.4.1
!pip install darts

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 636 kB 7.3 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 400 kB 7.6 MB/s 
     |████████████████████████████████| 43 kB 2.4 MB/s 
     |████████████████████████████████| 1.4 MB 51.4 MB/s 
     |████████████████████████████████| 9.8 MB 41.4 MB/s 
     |████████████████████████████████| 11.2 MB 28.0 MB/s 
     |████████████████████████████████| 179 kB 51.4 MB/s 
     |████████████████████████████████| 793 kB 51.7 MB/s 
     |████████████████████████████████| 585 kB 58.9 MB/s 
     |████████████████████████████████| 381 kB 56.6 MB/s 
     |████████████████████████████████| 944 kB 31.1 MB/s 
     |████████████

In [ ]:
#libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import ParameterGrid
from darts.timeseries import TimeSeries
from darts.utils.timeseries_generation import datetime_attribute_timeseries
from darts.dataprocessing.transformers import Scaler
from darts.models import RNNModel
from sklearn.metrics import mean_squared_error

/usr/local/lib/python3.7/dist-packages/distributed/config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)


In [ ]:
#load the data
#YYYY-MM-DD
df = pd.read_csv('nyc_data.csv', index_col = 0, parse_dates = True)
df.head()

,Demand,Easter,Thanksgiving,Christmas,Temperature,Marketing
Date,,,,,,
2015-01-01,720.000885,0,0,0,3.68,41.305
2015-01-02,581.276773,0,0,0,4.73,131.574
2015-01-03,754.117039,0,0,0,7.23,162.700
2015-01-04,622.252774,0,0,0,10.96,160.281
2015-01-05,785.373319,0,0,0,6.92,51.077


In [ ]:
#Rename variable
df = df.rename(columns = {'Demand': 'y'})
df.head(0)

,y,Easter,Thanksgiving,Christmas,Temperature,Marketing
Date,,,,,,


#Prepare for LSTM

In [ ]:
#Time Series object
series = TimeSeries.from_series(df.y)
covariates = TimeSeries.from_dataframe(df.iloc[:,1:])

In [ ]:
#year
year_series = datetime_attribute_timeseries(
    pd.date_range(start=series.start_time(),
                  freq = series.freq_str,
                  periods= df.shape[0]),
                  attribute ="year",
                  one_hot = False)

#month
month_series = datetime_attribute_timeseries(year_series,
                                            attribute ="month",
                                            one_hot = True)

#weekday
weekday_series = datetime_attribute_timeseries(year_series,
                                               attribute ="weekday",
                                               one_hot = True)

In [ ]:
# preparing scalers
transformer1 = Scaler()
transformer2 = Scaler()

In [ ]:
#Scale the Y
y_transformed = transformer1.fit_transform(series)

In [ ]:
# Scale the covariates
covariates = covariates.stack(year_series)
covariates_transformed = transformer2.fit_transform(covariates)
covariates_transformed = covariates_transformed.stack(month_series)
covariates_transformed = covariates_transformed.stack(weekday_series)

#LSTM

In [ ]:
#model
#https://unit8co.github.io/darts/generated_api/darts.models.forecasting.rnn_model.html
model = RNNModel(model = "LSTM",
                 hidden_dim = 20,
                 n_rnn_layers= 2,
                 dropout = 0,
                 n_epochs = 20,
                 optimizer_kwargs = {"lr": 0.003},
                 random_state = 1502,
                 training_length = 20,
                 input_chunk_length= 15,
                 pl_trainer_kwargs = {"accelerator": "gpu", "gpus": -1,
                                      "auto_select_gpus": True},
                 save_checkpoints = True)

In [ ]:
#fit the model to the data
model.fit(y_transformed,
          future_covariates = covariates_transformed)

2022-07-08 08:06:29 darts.models.forecasting.torch_forecasting_model INFO: Train dataset contains 2172 samples.
2022-07-08 08:06:29 darts.models.forecasting.torch_forecasting_model INFO: Time series values are 64-bits; casting model to float64.
2022-07-08 08:06:45 pytorch_lightning.trainer.connectors.accelerator_connector INFO: Auto select gpus: [0]
2022-07-08 08:06:45 pytorch_lightning.utilities.rank_zero INFO: GPU available: True, used: True
2022-07-08 08:06:45 pytorch_lightning.utilities.rank_zero INFO: TPU available: False, using: 0 TPU cores
2022-07-08 08:06:45 pytorch_lightning.utilities.rank_zero INFO: IPU available: False, using: 0 IPUs
2022-07-08 08:06:45 pytorch_lightning.utilities.rank_zero INFO: HPU available: False, using: 0 HPUs
2022-07-08 08:06:45 pytorch_lightning.accelerators.gpu INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
2022-07-08 08:06:45 pytorch_lightning.callbacks.model_summary INFO: 
  | Name          | Type             | Params
-----------------------------

Training: 0it [00:00, ?it/s]

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/callbacks/model_checkpoint.py:378: UserWarning: `ModelCheckpoint(monitor='val_loss')` could not find the monitored key in the returned metrics: ['train_loss', 'epoch', 'step']. HINT: Did you call `log('val_loss', value)` in the `LightningModule`?
  warning_cache.warn(m)


#Cross-validation

In [ ]:
#CV
cv = model.historical_forecasts(y_transformed,
                                future_covariates = covariates_transformed,
                                start = df.shape[0] - 180,
                                forecast_horizon = 31,
                                stride = 16,
                                retrain = True,
                                last_points_only = False)

In [ ]:
#store the results
rmse_cv = []
  
for i in range(len(cv)):
  #predictions
  predictions = TimeSeries.pd_series(transformer1.inverse_transform(cv[i]))

  #actuals
  start = predictions.index.min()
  end = predictions.index.max()
  actuals = df.y[start:end,]

  #compute the error
  error_cv = np.sqrt(mean_squared_error(actuals, predictions))

  #save the error
  rmse_cv.append(error_cv)

In [ ]:
np.mean(rmse_cv)

183.26426151744369

#Parameter Tuning 

In [ ]:
#Grid
param_grid = {'n_rnn_layers': [2,4],
              'hidden_dim': [20, 30],
              'dropout': [0, 0.2],
              'n_epochs': [10,20],
              'lr': [0.003],
              'training_length': [20],
              'input_chunk_length': [15]}
grid = ParameterGrid(param_grid)
len(list(grid))

16

In [ ]:
#Parameter tuning loop
rmse = []

#loop
for params in grid:
  #build the model
  model = RNNModel(model = "LSTM",  
                  hidden_dim = params['hidden_dim'],
                  n_rnn_layers= params['n_rnn_layers'],
                  dropout = params['dropout'],
                  n_epochs = params['n_epochs'],
                  optimizer_kwargs = {"lr": params['lr']},
                  random_state = 1502,
                  training_length = params['training_length'],
                  input_chunk_length= params['input_chunk_length'],
                  pl_trainer_kwargs = {"accelerator": "gpu", "gpus": -1,
                                        "auto_select_gpus": True},
                  save_checkpoints = True)


  #fit the model
  model.fit(y_transformed,
            future_covariates = covariates_transformed)

  #Cross-Validation
  cv = model.historical_forecasts(y_transformed,
                                  future_covariates = covariates_transformed,
                                  start = df.shape[0] - 180,
                                  forecast_horizon = 31,
                                  stride = 16,
                                  retrain = True,
                                  last_points_only = False)

  #Loop through the CV
  rmse_cv = []
    
  for i in range(len(cv)):
    #predictions
    predictions = TimeSeries.pd_series(transformer1.inverse_transform(cv[i]))

    #actuals
    start = predictions.index.min()
    end = predictions.index.max()
    actuals = df.y[start:end,]

    #compute the error
    error_cv = np.sqrt(mean_squared_error(actuals, predictions))

    #save the error
    rmse_cv.append(error_cv)

  #find the average error
  error = np.mean(rmse_cv)
  rmse.append(error)

In [ ]:
#Check the results
tuning_results = pd.DataFrame(grid)
tuning_results['rmse'] = rmse
tuning_results

In [ ]:
#export best parameters
best_params = tuning_results[tuning_results.rmse == tuning_results.rmse.min()].transpose()
best_params

,15
dropout,0.200000
hidden_dim,30.000000
input_chunk_length,15.000000
lr,0.003000
n_epochs,20.000000
n_rnn_layers,4.000000
training_length,20.000000
rmse,92.207997


In [ ]:
#get them
n_rnn_layers = int(best_params.loc['n_rnn_layers'])
hidden_dim = int(best_params.loc['hidden_dim'])
dropout = float(best_params.loc['dropout'])

#Parameter Tuning round 2

In [ ]:
#Grid
param_grid = {'n_rnn_layers': [n_rnn_layers],
              'hidden_dim': [hidden_dim],
              'dropout': [dropout],
              'n_epochs': [10,20],
              'lr': [0.001, 0.003],
              'training_length': [20, 30],
              'input_chunk_length': [15, 20]}
grid = ParameterGrid(param_grid)
len(list(grid))

16

In [ ]:
#Parameter tuning loop
rmse = []

#loop
for params in grid:
  #build the model
  model = RNNModel(model = "LSTM",  
                  hidden_dim = params['hidden_dim'],
                  n_rnn_layers= params['n_rnn_layers'],
                  dropout = params['dropout'],
                  n_epochs = params['n_epochs'],
                  optimizer_kwargs = {"lr": params['lr']},
                  random_state = 1502,
                  training_length = params['training_length'],
                  input_chunk_length= params['input_chunk_length'],
                  pl_trainer_kwargs = {"accelerator": "gpu", "gpus": -1,
                                        "auto_select_gpus": True},
                  save_checkpoints = True)


  #fit the model
  model.fit(y_transformed,
            future_covariates = covariates_transformed)

  #Cross-Validation
  cv = model.historical_forecasts(y_transformed,
                                  future_covariates = covariates_transformed,
                                  start = df.shape[0] - 180,
                                  forecast_horizon = 31,
                                  stride = 16,
                                  retrain = True,
                                  last_points_only = False)

  #Loop through the CV
  rmse_cv = []
    
  for i in range(len(cv)):
    #predictions
    predictions = TimeSeries.pd_series(transformer1.inverse_transform(cv[i]))

    #actuals
    start = predictions.index.min()
    end = predictions.index.max()
    actuals = df.y[start:end,]

    #compute the error
    error_cv = np.sqrt(mean_squared_error(actuals, predictions))

    #save the error
    rmse_cv.append(error_cv)

  #find the average error
  error = np.mean(rmse_cv)
  rmse.append(error)

In [ ]:
#Check the results
tuning_results = pd.DataFrame(grid)
tuning_results['rmse'] = rmse
tuning_results

In [ ]:
#export best parameters
best_params = tuning_results[tuning_results.rmse == tuning_results.rmse.min()].transpose()
best_params.to_csv("Forecasting Product/best_params_lstm.csv")